## 把网页保存到es中

In [1]:
import pymysql
# 创建连接
cnx = pymysql.connect(host='localhost', user='root', password='123qwe12')
cursor = cnx.cursor()

In [27]:
ids_douban = []
ids_page = []

def create_database():
    # 创建数据库
    cursor.execute("CREATE DATABASE IF NOT EXISTS IR_db")
    # 使用数据库
    cnx.select_db('IR_db')

def get_ids_from_douban():
    # 创建SQL查询
    sql = "SELECT id FROM douban"
    # 执行查询
    cursor.execute(sql)
    # 获取所有的结果
    results = cursor.fetchall()
    # 将结果从元组列表转换为普通列表
    ids_douban = [result[0] for result in results]
    return ids_douban

def get_ids_from_same_title():
    # 创建SQL查询
    sql = "SELECT page_id FROM same_title"
    # 执行查询
    cursor.execute(sql)
    # 获取所有的结果
    results = cursor.fetchall()
    # 将结果从元组列表转换为普通列表
    ids_same_title = [result[0] for result in results]
    return ids_same_title

def get_ids_from_page():
    global ids_page
    # 创建SQL查询
    sql = "SELECT id FROM page"
    # 执行查询
    cursor.execute(sql)
    # 获取所有的结果
    results = cursor.fetchall()
    # 将结果从元组列表转换为普通列表
    ids = [result[0] for result in results]
    # 获取相同title的id
    ids_same_title = get_ids_from_same_title()
    print(len(ids_same_title))
    # 去除相同title的id
    for id in ids:
        if id not in ids_same_title:
            ids_page.append(id)
    # 从same_title中按title分组，获取每组的第一个id
    sql = "SELECT MIN(page_id) FROM same_title GROUP BY title"
    cursor.execute(sql)
    results = cursor.fetchall()
    ids = [result[0] for result in results]
    # 将结果添加到ids_page中
    ids_page.extend(ids)

def get_content(table, id):
    # 创建SQL查询
    sql = "SELECT * FROM {} WHERE id = %s".format(table)
    # 执行查询
    cursor.execute(sql, (id,))
    # 获取结果
    result = cursor.fetchone()
    # 如果结果不为空，将结果转换为字典并返回，否则返回None
    if result:
        return {
            'id': result[0],
            'title': result[1],
            'content': result[2],
            'url': result[3]
        }
    else:
        return None

create_database()
ids_douban = get_ids_from_douban()
print(len(ids_douban))
get_ids_from_page()
print(len(ids_page))


5374
1435
5699


### 创建es索引

In [1]:
from elasticsearch import Elasticsearch

# 实例化es
es = Elasticsearch(hosts="http://localhost:9200")

#es.indices.create(index='web')

### 读取数据插入web

In [28]:
global_id = 1
def insert_douban_web():
    global global_id
    for id in ids_douban:
        # 获取内容
        content = get_content('douban', id)
        doc = {
            'web_id': content['id'],
            'title': content['title'],
            'content': content['content'],
            'url': content['url'],
            'type': 'douban'
        }
        # 插入数据
        es.create(index='web', id = global_id, document=doc)
        global_id += 1

def insert_page_web():
    global global_id
    for id in ids_page:
        # 获取内容
        content = get_content('page', id)
        doc = {
            'web_id': content['id'],
            'title': content['title'],
            'content': content['content'],
            'url': content['url'],
            'type': 'page'
        }
        # 插入数据
        es.create(index='web', id = global_id, document=doc)
        global_id += 1    

In [29]:
global_id = 1
insert_douban_web()
print('douban done', global_id)
insert_page_web()
print('page done', global_id)

douban done 5375
page done 11074


In [24]:
es.indices.delete(index='web')

ObjectApiResponse({'acknowledged': True})

### 测试查询数据

In [30]:
response = es.count(index='web')
print('Document count:', response['count'])

Document count: 11073


In [2]:
def deal_bool_query(query):
    # 执行搜索
    response = es.search(
        index="web",
        body={
            "query": {
                "bool": {
                    "must": [
                        {
                            "multi_match": {
                                "query": query,
                                "fields": ["title", "content"]
                            }
                        }
                    ]
                }
            }
        }
    )
    # 获取搜索结果
    hits = response["hits"]["hits"]
    # 格式化搜索结果
    results = [
        {
            "id": hit["_id"],
            "title": hit["_source"]["title"],
            "content": hit["_source"]["content"],
            "url": hit["_source"]["url"],
            "type": hit["_source"]["type"]
        }
        for hit in hits
    ]
    return results

In [4]:
results = deal_bool_query('肖申克的救赎')
for result in results:
    print(result['title'], result['url'], result['type'])

肖申克的救赎 https://book.douban.com/subject/1829226/ douban
十年·肖申克的救赎 https://movie.douban.com/review/1000369/ douban
肖申克的救赎 The Shawshank Redemption https://movie.douban.com/subject/1292052/ douban
《肖申克的救赎》到底“救赎”了什么？ https://movie.douban.com/review/10350620/ douban
肖申克的救赎，读书笔记 https://book.douban.com/review/1336253/ douban
《肖申克的救赎》的一些幕后花絮 https://movie.douban.com/review/1062920/ douban
为何《肖申克的救赎》在IMDb和豆瓣都能排第一？ https://movie.douban.com/review/9259304/ douban
汲汲而生，汲汲而死 https://book.douban.com/review/1597365/ douban
《肖申克的救赎》：1994—2007，希望就是现实 https://movie.douban.com/review/1127585/ douban
关于“救赎” https://movie.douban.com/review/8848890/ douban
